In [ ]:
%pip install flask tensorflow resampy librosa opencv-python pydub matplotlib

In [2]:
import tensorflow as tf

In [3]:
# Load the trained model
model = tf.keras.models.load_model('./model/cnn.keras')


In [ ]:
model.summary()


In [6]:
from numpy import ndarray
import numpy as np
import librosa
import matplotlib.pyplot as plt
import cv2 as cv
import io
import base64

HOP_LENGTH = 512        # number of samples between successive frames
WINDOW_LENGTH = 512     # length of the window in samples
N_MEL = 128             # number of Mel bands to generate


def spectrogram_fixed_length(audio, rate, total_samples = 128) -> ndarray:
    spectrogram = librosa.feature.melspectrogram(
        y = audio, 
        sr=rate, 
        hop_length=HOP_LENGTH, 
        win_length=WINDOW_LENGTH,
        n_fft=512
    )

    spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)

    spectrogram_length = spectrogram_db.shape[1]

    if spectrogram_length != total_samples:
        spectrogram_db = librosa.util.fix_length(
            spectrogram_db, size=total_samples, axis=1,
        )
    
    return spectrogram_db


def generate_spectrogram(audio, sample_rate):
    spectrogram = spectrogram_fixed_length(audio, sample_rate)
    librosa.display.specshow(spectrogram, cmap='viridis')
    plt.axis('tight')
    plt.axis('off')
    plt.tight_layout()
    plt.savefig("uploads/spectrogram.png", bbox_inches="tight", pad_inches=0.0)



# TODO: use sliding window
def preprocess_audio(audio):
    DURATION = 3
    audio, sample_rate = librosa.load(audio, duration=DURATION, res_type='kaiser_fast')
    generate_spectrogram(audio, sample_rate)
   
    IMG_SIZE = 256

    img_array = cv.imread("uploads/spectrogram.png")
    new_array = cv.resize(img_array, (IMG_SIZE, IMG_SIZE))

    print("Done!")

    return np.array(new_array).reshape(-1, IMG_SIZE, IMG_SIZE, 3) / 255


In [ ]:
from flask import Flask, render_template, request, session

# import os
# from pydub import AudioSegment

app = Flask("Audio Classifier")
app.config['UPLOAD_FOLDER'] = 'uploads/'
app.secret_key = '123456'

LABELS_OUTER = [
    "air_conditioner",
    "car_horn",
    # "children_playing",
    # "dog_bark",
    "drilling",
    "engine_idling",
    "gun_shot",
    "jackhammer",
    # "siren",
    "street_music"
]

@app.route('/', methods=['GET', 'POST'])
def index():
    threshold = int(session.get('threshold', 0))
    selected_categories = [int(x) for x in session.get('selected_categories', [])]


    return render_template(
        './index.html', 
        categories=LABELS_OUTER,
        threshold=threshold, 
        selected_categories=selected_categories
    )


@app.route('/process_categories', methods=['POST'])
def process_categories():
    selected_categories = request.get_json().get("selected_categories", [])
    # convert them to integers
    selected_categories = [int(x) for x in selected_categories]

    threshold = request.get_json().get('threshold', 50)

    session['selected_categories'] = selected_categories
    session['threshold'] = threshold

    return {"success": True}

LABELS = [
    "air_conditioner",
    "car_horn",
    "children_playing",
    "dog_bark",
    "drilling",
    "engine_idling",
    "gun_shot",
    "jackhammer",
    "siren",
    "street_music"
]

@app.route('/classify', methods=['POST'])
def classify():
    # Get the audio file from the request
    audio_file = request.files['audio_file']

    # Process the audio file using your trained model
    audio_data = preprocess_audio(audio_file)
    
    print("predicting...")
        
    prediction = model.predict(audio_data)
        

    print(prediction)

    # categories to include
    selected_categories = session.get('selected_categories', [])

    print(selected_categories)
    new_selected = []
    for selected in selected_categories:
        if selected < 2:
            new_selected.append(selected)
        elif selected == 6:
            new_selected.append(9)
        else:
            new_selected.append(selected + 2)

    selected_categories = new_selected


    confidence_threshold = float(session.get('threshold', 0.))
    print(selected_categories)

    p = {}
    pred = prediction.flatten()

    print(pred)


    for (index, label) in enumerate(LABELS):
        if index not in selected_categories:
            continue

        confidence = float(pred[index] * 100.0)

        if confidence < confidence_threshold:
            continue

        p[label] = confidence

    print("labels:", p)

    

    print(confidence_threshold)


    # create a predicted label
    if len(p) == 0:
        if len(selected_categories) == 0:
            predicted = "None Found. You must select a nuisance."
        else:
            predicted = "None Found. Try lowering the detection threshold."
    else:
        predicted = max(p, key=p.get)


    # create spectrogram image
    file = open("uploads/spectrogram.png", 'rb')
    image_buffer = file.read()
    spectrogram = base64.b64encode(image_buffer).decode('utf-8')

    
    # mp3_bytes = io.BytesIO()
    # audio = AudioSegment.from_file(audio_file.read())
    # audio.export(mp3_bytes, format='mp3')
    # # mp3_data = mp3_bytes.getvalue()
    
    # audio_data = base64.b64encode(mp3_bytes.read()).decode('utf-8')
    
    # Return the classification result
    return {
        'prediction': predicted,
        'spectrogram': spectrogram, 
        # 'audio': audio_data,
        # 'audio_mime': audio_file.mimetype,
        'predictions': p,
    }

app.run()


 * Serving Flask app 'Audio Classifier'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit

Done!
predicting...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step


127.0.0.1 - - [28/Apr/2025 12:53:15] "POST /classify HTTP/1.1" 200 -


[[1.5942263e-04 4.5501636e-04 1.7874947e-06 1.7318914e-06 2.5052982e-03
  3.4416016e-04 9.9638063e-01 1.0392582e-04 1.8025385e-06 4.6167770e-05]]
[0, 1, 2, 3, 4, 5, 6, 7]
[0, 1, 4, 5, 6, 7, 9, 9]
[1.5942263e-04 4.5501636e-04 1.7874947e-06 1.7318914e-06 2.5052982e-03
 3.4416016e-04 9.9638063e-01 1.0392582e-04 1.8025385e-06 4.6167770e-05]
labels: {'air_conditioner': 0.015942263416945934, 'car_horn': 0.04550163575913757, 'drilling': 0.2505298238247633, 'engine_idling': 0.03441601584199816, 'gun_shot': 99.6380627155304, 'jackhammer': 0.010392582044005394, 'street_music': 0.004616777005139738}
0.0


...

[[4.7957462e-01 1.0397708e-06 2.4029294e-12 7.3960820e-12 3.0438410e-04
  6.9158850e-05 2.0446669e-09 1.5836726e-05 7.3771180e-12 5.2003497e-01]]
[0, 1, 2, 3, 4, 5]
[0, 1, 4, 5, 6, 7]
[4.7957462e-01 1.0397708e-06 2.4029294e-12 7.3960820e-12 3.0438410e-04
 6.9158850e-05 2.0446669e-09 1.5836726e-05 7.3771180e-12 5.2003497e-01]
labels: {'air_conditioner': 47.95746207237244}
2.0


127.0.0.1 - - [28/Apr/2025 13:01:37] "POST /process_categories HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2025 13:01:38] "POST /process_categories HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2025 13:01:45] "POST /process_categories HTTP/1.1" 200 -


Done!
predicting...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step


127.0.0.1 - - [28/Apr/2025 13:01:47] "POST /classify HTTP/1.1" 200 -


[[4.7957462e-01 1.0397708e-06 2.4029294e-12 7.3960820e-12 3.0438410e-04
  6.9158850e-05 2.0446669e-09 1.5836726e-05 7.3771180e-12 5.2003497e-01]]
[0, 1, 2, 3, 4, 5, 6]
[0, 1, 4, 5, 6, 7, 9]
[4.7957462e-01 1.0397708e-06 2.4029294e-12 7.3960820e-12 3.0438410e-04
 6.9158850e-05 2.0446669e-09 1.5836726e-05 7.3771180e-12 5.2003497e-01]
labels: {'air_conditioner': 47.95746207237244, 'car_horn': 0.00010397708365417202, 'drilling': 0.030438409885391593, 'engine_idling': 0.0069158850237727165, 'gun_shot': 2.0446668802520662e-07, 'jackhammer': 0.0015836725651752204, 'street_music': 52.00349688529968}
0.0
